# AI Detector - Export Model

Since, we have trained, evaluated, and did inference on our model, it's time to export it for deployment!

## 1. Import Necessary Libraries
At first, let's import necessary libraries to convert our model to ONNX format.

In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
from onnxruntime.quantization import quantize_dynamic, QuantType
import os

## 2. Define `export_to_onnx()` function
- **Params:**
  - `model_path` -> fine-tuned model path
  - `onnx_path` -> converted ONNX model path

This function converts the fine-tuned SBERT model to ONNX format.

In [9]:
def export_to_onnx(model_path, onnx_path):
    # Load the SentenceTransformer model
    model = AutoModel.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Prepare dummy input (adjust input size as needed)
    inputs = tokenizer("Candidate coding answer", "AI generated answer", padding=True, truncation=True, return_tensors="pt")

    # Export the model
    torch.onnx.export(
        model,
        # Model input (tuple)
        args=(inputs['input_ids'], inputs['attention_mask']),
        f=onnx_path,  # Output ONNX file path
        input_names=["input_ids", "attention_mask"],
        output_names=["output"],  # Name of the model output
        dynamic_axes={"input_ids": {0: "batch_size", 1: "sequence_length"}, 
                    "attention_mask": {0: "batch_size", 1: "sequence_length"}, 
                    "output": {0: "batch_size"}},
        opset_version=11,
    )

    print(f"Model exported to {onnx_path}")

## 3. Define `quantize_onnx_model()` Function

- **Params:**
  - `onnx_path` -> Exported ONNX model path
  - `quantized_onnx_path` -> Compressed ONNX model path

This function compresses and optimizes the ONNX for deployment.

In [10]:
def quantize_onnx_model(onnx_path, quantized_onnx_path):
    quantize_dynamic(onnx_path,
                     quantized_onnx_path,
                     weight_type=QuantType.QUInt8)  # 8-bit integers for weights
    print(f"Quantized model saved to: {quantized_onnx_path}")

## 4. Convert / Export the Model

At first, define the fine-tuned model for deployment.

In [11]:
model_name = "all-mpnet-base-v2"
# model_name = "all-distilroberta-v1"
# model_name = "all-MiniLM-L12-v2"
# model_name = "all-MiniLM-L6-v2"
# model_name = "multi-qa-mpnet-base-dot-v1"

Now, define the necessary export directories.

In [12]:
# Load the model
model_dir = data_dir = os.path.join(
    os.path.abspath(''), os.pardir, 'models')
model_path = os.path.join(model_dir, f"fine-tuned_{model_name}")

onnx_path = os.path.join(model_dir, f"fine-tuned_{model_name}.onnx")

quantize_onnx_path = os.path.join(
    model_dir, f"fine-tuned_{model_name}_quantized.onnx")


Convert to ONNX

In [13]:
export_to_onnx(model_path, onnx_path)

Model exported to e:\Data Science\AI-Detector\notebooks\..\models\fine-tuned_all-mpnet-base-v2.onnx


Compress the ONNX model

In [14]:
quantize_onnx_model(onnx_path, quantize_onnx_path)

Quantized model saved to: e:\Data Science\AI-Detector\notebooks\..\models\fine-tuned_all-mpnet-base-v2_quantized.onnx
